In [26]:
!pip install streamlit
!pip install pyPDF2
!pip install langchain
!pip install faiss-cpu
!pip install transformers
!pip install google-generativeai
!pip install python-dotenv
!pip install langchain-google-genai

In [27]:
!pip install pyPDF2

In [28]:
!pip install langchain

In [29]:
!pip install faiss-cpu

In [30]:
!pip install transformers

In [31]:
!pip install google-generativeai

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.8 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [32]:
!pip install python-dotenv

In [33]:
!pip install langchain-google-genai

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [34]:
from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores.faiss import FAISS
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import os
import requests
from huggingface_hub import configure_http_backend
import streamlit as st
from pathlib import Path
import shutil

In [35]:
# fixing the SSL issue
def backend_factory() -> requests.Session:
    session = requests.Session()
    session.verify = False
    return session

configure_http_backend(backend_factory=backend_factory)

In [36]:
# set GEMINI key
os.environ["GOOGLE_API_KEY"] = " "

In [37]:
# embeddings and model
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

In [38]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [39]:
# prompt
prompt_temp = PromptTemplate.from_template(
    """
    You are an intelligent assistant designed to answer questions based on the content of uploaded PDF documents.
    Please provide accurate and helpful answers to the questions asked, using the context provided from the documents.

    Context:
    {context}

    Question:
    {question}

    Helpful Answer:
    """
)

prompt_str = prompt_temp.template
prompt = PromptTemplate(template=prompt_str, input_variables=["history", "context", "question"])

# STREAMLIT UI
st.title("File Upload and Query")
uploaded_file = st.file_uploader("Choose a file", type=["pdf"])
query = st.text_input("Enter your query")


2025-08-04 07:49:18.597 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:49:18.598 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:49:18.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:49:18.602 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:49:18.604 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:49:18.607 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:49:18.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:49:18.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [40]:
# loading data
def load_data(uploaded_file):
    save_path = os.path.join("./data/", uploaded_file.name)
    with open(save_path, "wb") as f:
        f.write(uploaded_file.getbuffer())
    loader = PyPDFDirectoryLoader("./data/")
    docs = loader.load()
    return docs

In [41]:
# splitting text
def split_text(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=64)
    return text_splitter.split_documents(docs)


In [44]:
# storing in vector DB
def store_VDB(texts):
    global qa_chain
    global memory


    # Delete old vectorstore
    db_path = Path("./vectorstores/db_faiss")
    if db_path.exists() and db_path.is_dir():
        shutil.rmtree(db_path)  # Removes old vectorstore folder

    vectorstore = FAISS.from_documents(documents=texts, embedding=embeddings_model)
    vectorstore.save_local(str(db_path))

    # Rebuild memory
    memory = ConversationBufferMemory(
        memory_key="history",
        input_key="question")

In [46]:
#storing in vector db
def store_VDB(texts):
    global qa_chain
    global memory

    # Delete old vectorstore
    db_path = Path("./vectorstores/db_faiss")
    if db_path.exists() and db_path.is_dir():
        shutil.rmtree(db_path)  # Removes old vectorstore folder

    vectorstore = FAISS.from_documents(documents=texts, embedding=embeddings_model)
    vectorstore.save_local(str(db_path))

    # Rebuild memory
    memory = ConversationBufferMemory(
        memory_key="history",
        input_key="question"
    )

    # Load new DB
    db = FAISS.load_local(str(db_path), embeddings_model, allow_dangerous_deserialization=True)

    # QA Chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=model,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True,
        chain_type_kwargs={
            "verbose": True,
            "prompt": prompt,
            "memory": memory,
        },
    )

    return db

In [47]:
# defining QA chain
def create_chain(vectorstore):
    return RetrievalQA.from_chain_type(
        llm=model,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        return_source_documents=True,
        chain_type_kwargs={
            "verbose": True,
            "prompt": prompt,
            "memory": ConversationBufferMemory(memory_key="history", input_key="question"),
        },
    )

# answering query
def query_qa_chain(qa_chain, query):
    response = qa_chain({"query": query})
    return response['result']


In [48]:
# main logic
if st.button("Get Answer"):
    if uploaded_file is not None and query:
        docs = load_data(uploaded_file)
        texts = split_text(docs)

        db_path = Path("./vectorstores/db_faiss")
        db = store_VDB(texts)
        qa_chain = create_chain(db)
        result = query_qa_chain(qa_chain, query)
        st.write("Answer:", result)
    else:
        st.write("Please upload a file and enter a query.")

2025-08-04 07:51:51.886 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:51:51.889 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:51:51.890 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:51:51.892 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:51:51.894 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 07:51:51.895 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [50]:
#streamlit run app.py